# AI Assisted Annotation 

Manual annotation is slow, tedious and costly. Faster labeling of 3D volumes using AI annotation models accelerates this process. Clara Train offers an AIAA API that easily integrates into common medical imaging viewers
<br><img src="screenShots/AIAASpeedup.png" alt="Drawing" style="height: 400px;"/><br>

AIAA is based on a server client model as shown below 
<br><img src="screenShots/AIAAClientServer.png" alt="Drawing" style="height: 400px;"/><br>



By the end of this notebook you will be able to:
- Start AIAA server 
- Load a deep grow model
- Annotate using deep grow 
- Load your model and use it for annotations  
- Stop AIAA server


## Prerequisites
- Nvidia GPU with 8Gb of memory   


### Resources
It might be helpful to watch the GTC Digital 2021 talk on Clara Train SDK
- [Clara Train 4.0 - 101 Getting Started [SE2688]](https://gtc21.event.nvidia.com/media/Clara%20Train%204.0%20-%20101%20Getting%20Started%20%5BSE2688%5D/1_0qgfrql2) 
Clara train Getting started: cover basics, BYOC, AIAA, AutoML 


# Lets get started

To interact with AIAA from the command line you should use the AIAA as `AIAA -h`. 
This would show you all commands as shown below
```
Usage: AIAA [-h] {start,stop,status,list,load,delete} ...

commands:
  start   Start AIAA Server
          AIAA start -h
  stop    Stop AIAA Server
  status  Check if AIAA Server is running
  list    List all models loaded in AIAA
  logs    Fetch AIAA Logs if server is running <lines>
  load    Load a model to AIAA using one of the sources {ngc|mmar|zip|config}
          sources:
            ngc      Load model from NGC
                     AIAA load <model> ngc <path> <version>
            mmar     Load model from MMAR Folder
                     AIAA load <model> mmar <path>
            zip      Load model from ZIP Archive
                     AIAA load <model> zip <path>
            config   Load model using AIAA Config and Model saved file
                     AIAA load <model> config <aiaa_config> <saved_model>
            pipeline Load pipeline (as vitual model) using AIAA Config
                     AIAA load <model> config <aiaa_config>
  delete  Delete a model from AIAA
          AIAA delete <model>
```

Before we get started let us check that we have an NVIDIA GPU available in the docker by running the cell below

In [ ]:
# following command should show all gpus available 
!nvidia-smi


# 1. Start AIAA server
First lets set up AIAA path and change some permissions 

In [ ]:
AIAA_ROOT="/claraDevDay/AIAA/workspace/"
AIAA_PORT="5000"
!mkdir -p $AIAA_ROOT
!chmod 777 $AIAA_ROOT
print ("AIAA_ROOT is set to ",AIAA_ROOT)


##  1.2 AIAA Server with Triton BackEnd 
Starting V4 Triton server was moved out of the clara train container. 
In order to use Triton as the AIAA backend, you should use `restartClaraTrain.sh`. 
The script uses docker-compose to start both the clara train container and triton container, 
while connected them through the docker internal network. 

Now lets start the AIAA server by running the cell below

In [ ]:
! AIAA start -b -w $AIAA_ROOT --engine TRITON --triton_ip tritonserver

# 2 Check on AIAA Server 

## 2.1 using CLI
You should check on the AIAA status

In [ ]:
!AIAA status   

You can also get the last 15 lines of the logs use

In [ ]:
!AIAA logs 15 

You can then easily filter on errors as  

In [ ]:
!AIAA logs |grep errors

To list models you can run 

In [ ]:
!AIAA list


### 2.2 Using Browser
You can also check on the AIAA server using your browser by checking:
- Main url `http://localhost:5000/` <br>
- APIS to list, upload, delete models `http://localhost:5000/docs/`
- Check logs `http://localhost:5000/logs`
- List available models `http://localhost:5000/v1/models/` 
 

### 2.3 (For advanced users) Using curl commands
You can also use curl command to do list logs 
The cell below will get the last 15 lines of the logs. 

In [ ]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/logs/?lines=15"
!curl -X GET $http_str -H "accept: application/json" 

To list models

In [ ]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/v1/models"
!curl -X GET $http_str -H "accept: application/json"

#  3. Load models in AIAA Server. 
For this notebook we can download models from [NGC](). Models on NGC are either:
- Annotation Models <br>
[Deep Extreme Cut: From Extreme Points to Object Segmentation](https://arxiv.org/abs/1711.09081)
<br><img src="screenShots/AIAAAnnotation.png" alt="Drawing" style="height: 400px;"/><br>

- Segmentation Models
- DeepGrow MMAR 
This is an interactive model to get you started with annotation. CNN takes in single channel (image) + use single click 
for foreground or background location then produces the segmentation. it is based on [Interactive segmentation of medical images through
fully convolution neural networks](https://arxiv.org/pdf/1903.08205.pdf)
<br><img src="screenShots/AIAADeepGrow.png" alt="Drawing" style="height: 400px;"/><br>


## 3.0 Loading options
In order to load a model to AIAA we will use the AIAA cli as 

In [ ]:
! AIAA load -h

This will show you options which allows to load from ngc or a local MMAR or a local zip file, etc 
```
root@claratrain:/claraDevDay/AIAA# AIAA load -h
Loading Model: -h
Usage: AIAA load <model> {ngc|mmar|zip|config|pipeline} ...
```

## 3.1 Load model trained from Getting started notebook
In order to load model trained in the getting started notebook you can run

In [ ]:
modelName="Test"
modelFolderPath="/claraDevDay/GettingStarted/models/config_train_Unet/model.ts"
configFolderPath="/claraDevDay/GettingStarted/config/config_aiaa.json"
!AIAA load $modelName config $configFolderPath $modelFolderPath 


## 3.2 Using NGC CLI to download model from NGC (Will NOT work for EA) 
You can see a [list of available pre-trained models](https://ngc.nvidia.com/containers/nvidia:clara-train-sdk) on NGC. 
You can also use `ngc registry model list nvidia/med/clara_*` to get a list of models.

In [ ]:
!ngc registry model list nvidia/clara_*
!ngc registry model list nvidia/med/clara_*


The cell below will download the the deep grow model from NGC
***Will Not work for EA***

The following cell will download the spleen model and load it into the AIAA server

## 3.3 Manually Download model from NGC 
If you have a private registry or have early access (EA) to nvidia's developer program, 
you can manually downloaded the MMAR form NGC by clicking the ... button as shown below
<br><img src="screenShots/DownloadFromNGC.png" alt="Drawing" style="height: 400px;"/><br>

**You should download zip file to /claraDevDay/AIAA/DownloadsFromNGC/**
 


## 3.4 This section should be deleted / updated Check for models loaded into AIAA
Zip file would have the MMAR structure. 
Cell below would unzip the MMAR then upload it to AIAA server 

In [ ]:
downloadedFile="GA_deepgrow_3d.zip"

downloadDir=AIAA_ROOT+"../DownloadsFromNGC/"
modelName=downloadedFile[:-4]
modelFolderPath=downloadDir+modelName

%cd $downloadDir
!unzip $downloadedFile -d $modelName

In [ ]:
!AIAA load $modelName mmar $modelFolderPath


## 3.5 load a pipeline created from multiple models into AIAA
AIAA now supports the concept of a pipelines which triggers multiple models one at a time.
cell below creates a pipeline out of 3D deepgrow which gets new seed points for multiple runs of deepgrow 2D.

In [ ]:
modelName="GA_Deepgrow_Pipeline"
configFolderPath=AIAA_ROOT+"../Deepgrow2D3D_pipeline.json"
 
!AIAA load $modelName pipeline $configFolderPath 



## 3.6 Check for models loaded into AIAA
Let us check the server and check that the model is uploaded. 
Cell below would list models loaded by the AIAA server 

In [ ]:
!AIAA list 

## 3.7 (Optional for advanced users) Loading models using curl commands
For advanced users or programmers integrating with AIAA, 
They could do the above task using curl commands as:  

```
http_str="http://127.0.0.1:"+AIAA_PORT+"/admin/model/"+modelName
conf_str="config=@"+downloadDir+modelName+"/config/config_aiaa.json;type=application/json"
dataArg="data=@"+downloadDir+modelName+"/models/model.ts"
cmd='curl -X PUT "'+http_str +'" -F "'+conf_str +'" -F "'+dataArg +'"'
```
Command below would download from NGC and load the deep grow and spleen segmentation model to AIAA
```
# model name from NGC is clara_train_deepgrow_aiaa_inference_only
http_str="http://127.0.0.1:"+AIAA_PORT+"/admin/model/clara_deepgrow"
!curl -X PUT $http_str \
     -H "accept: application/json" \
     -H "Content-Type: application/json" \
     -d '{"path":"nvidia/med/clara_train_deepgrow_aiaa_inference_only","version":"1"}'
```
```
http_str="http://127.0.0.1:"+AIAA_PORT+"/admin/model/clara_ct_seg_spleen"
!curl -X PUT $http_str \
     -H "accept: application/json" \
     -H "Content-Type: application/json" \
     -d '{"path":"nvidia/med/clara_ct_seg_spleen","version":"1"}'
```


# 4. AIAA Clients 

AIAA server can connect to any client that implements the APIs found [here](https://github.com/NVIDIA/ai-assisted-annotation-client
). 
NVIDIA has already implemented these APIs for a number of open source viewers as:


## 4.1. [3D Slicer](https://www.slicer.org/)

In order to use slicer you should: 
1. Install and setup slicer 3d following steps [here](https://github.com/NVIDIA/ai-assisted-annotation-client/tree/master/slicer-plugin)
    1. Download and install recent 3D Slicer Preview Release (4.11.x) from [here](http://download.slicer.org/).
    For Early access please use slicer 4.13 (unstable release) this is needed to enable 3d geepgrow models. 
    2. Start 3D Slicer and open the Extension manager
    3. Install NvidiaAIAssistedAnnotation extension (in Segmentation category), wait for the installation to complete, and click Restart
2. Install by searching for nvidia in the plugin manager
3. Configure plugin. Add the AIAA server location and make sure the session is enabled
   <br><img src="screenShots/SlicerConfig.png" alt="Drawing" style="height: 400px;"/>
4. You should load a volume and start trying the spleen and deep grow model as shown below
     
<br><img src="screenShots/Slicer.png" alt="Drawing" style="height: 400px;"/><br>

## 4.2. [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK))
MITK is another viewer that you can use with AIAA. 
You can download and install it [here](http://mitk.org/wiki/Downloads). 
Please make sure you install the release with nvidia AIAA.<br>
_Note_: Deep grow is not enabled yet in MITK 


## 4.3. [OHIF](https://docs.ohif.org/)
We have integrated AIAA with OHIF as a plugin, This has also been adapted in to XNAT. 
Plugin code can be found as a [branch of OHIF github](https://github.com/SachidanandAlle/Viewers). 

For more on how to use OHIF please see [OHIF Notebook](AIAAwOHIF.ipynb)

# 5. Delete Models from AIAA server  

In [ ]:
# if you need to change the model name 
#modelName="Deepgrow3DV4EA"
!AIAA delete $modelName


Or use the http API
```
http_str='"http://127.0.0.1:'+AIAA_PORT+'/admin/model/'+modelName+'"'
!curl -X DELETE $http_str
```


# 6. Stop AIAA server
In order to stop the AIAA server you can run cell below.

In [ ]:
!AIAA stop


However, this will **not free up your gpu memory** 
since we are using **triton which is running in another container.**
In order to release the gpu memory you would need to either stop the triton container, 
or delete the models from the triton model directory `/claraDevDay/AIAA/workspace/triton_models/`

In [ ]:
#!mkdir $AIAA_ROOT/triton_models_full
#!mv $AIAA_ROOT/triton_models_full_back/* $AIAA_ROOT/triton_models_full/ 